# Preprocessor Notebook : Zipped CSV Files

Ce notebook permet de traiter les sources de données récupérés en .zip et contenant des fichiers csv.

 ### Paramètres
 Ce Notebook prend des paramètres en entrée, définis sur la seconde cellule (ci-dessous).
 La cellule a le tag "parameters" ce qui permet de lui passer des valeurs via papermill.
 - filepath : le chemin vers le fichier Excel à traiter
 - model_name : le nom du modèle source

 ### Principe
 Ce notebook extrait les fichiers CSV contenus dans un fichier ZIP et les charge en schéma Bronze.
 Chaque fichier CSV constitue un dataset et donc une table en schéma Bronze. 
 La règle de nommage des tables est la suivante :
 - Par défaut : "nom de la table" = "nom_domaine"_"nom_modèle"_"nom du fichier csv extrait"
 - Si le nom dépasse 63 caractères (limite PostgreSQL) : "nom de la table" = "nom_domaine"_"nom du fichier csv extrait"
 
 La suite d'actions réalisée est :
 - Import des fonctions utiles
 - Dézip de tous les fichiers contenus dans le .zip
 - Dump d'un artefact pour chaque fichier extrait (quel que soit le format)
 - Import de tous les artefacts CSV en Pandas 
 - Initialisation d'une connexion PostGreSQL
 
 Pour chaque dataframe :
 - Drop de la table Bronze (if exists)
 - Chargement du dataframe en Bronze

In [1]:
# Manage all imports
import pandas as pd
import os
import datetime
import sys

# Dirty trick to be able to import common odis modules, if the notebook is not executed from 13_odis
current_dir = os.getcwd()
parent_dir = os.path.dirname(os.getcwd())
while not current_dir.endswith("13_odis"):
    print("changing to parent dir")
    os.chdir(parent_dir)
    current_dir = parent_dir
    parent_dir = os.path.dirname(current_dir)

print(os.getcwd())
sys.path.append(current_dir)

changing to parent dir
/Users/alex/dev/13_odis


# Initialisation

Chargement des principaux imports et variables

In [2]:
# additional imports
from common.config import load_config
from common.data_source_model import DataSourceModel
from common.utils.file_handler import FileHandler
from common.utils.interfaces.data_handler import OperationType

In [3]:
# define parameters for papermill

# model_name = "emploi.salaire_median"
# filepath = 'data/imports/emploi/emploi.salaire_median_1.zip'

model_name = "population.nb_menages_2021"
filepath = 'data/imports/population/population.nb_menages_2021_1.zip'

# model_name = "emploi.etablissements_employeurs_secteur_prive"
# filepath = 'data/imports/emploi/emploi.etablissements_employeurs_secteur_prive_1.zip'

In [4]:
# Initialize common variables
dataframes = {}
artifacts = []

config = load_config("datasources.yaml", response_model=DataSourceModel)
model = config.get_model( model_name = model_name )
start_time = datetime.datetime.now()

# Instantiate File Handler for file loads and dumps
handler = FileHandler()

# Extraction des CSV

Extraction des fichiers CSV du Zip, avec une fonction pour calculer les noms de fichier

In [ ]:
def get_subtable_name(dfname:str, model:DataSourceModel) -> str:

    subtable_name = ''

    long_name = f"{model.table_name}_{dfname.lower()}"

    print(len(long_name))

    if len(long_name) >= 63:
        print(f"Long table name exceeds the 63 characters limit: {long_name}")
        subtable_name = f"{model.domain_name}_{dfname.lower()}"
        print(f"Creating table with shorter name: {subtable_name}")
        print(f"Shorter table name length: {len(subtable_name)}")
    else:
        subtable_name = long_name

    return subtable_name

In [6]:
from zipfile import ZipFile

# unzip and dump files into the data/imports folder
with open(filepath, 'rb') as f:
    zip_archive = ZipFile(f)

    zip_members = zip_archive.infolist()
    for member in zip_members:

        if not member.is_dir():

            member_filename = member.filename
            member_name = member_filename.split(".")[0]
            member_format = member_filename.split(".")[-1]
            print(member_format)
            
            f_member = zip_archive.open( member, 'r' ).read()
    
            artifact = handler.artifact_dump(
                f_member,
                get_subtable_name(member_name,model),
                model,
                format = member_format
            )

            print(artifact.model_dump(mode="yaml"))

            artifacts.append(artifact)

csv
39
2025-06-25 23:16:07,360 - DEBUG :: file_handler.py :: dump (130) :: dumping: data/imports/population/population.nb_menages_2021_population_nb_menages_2021_td_men1_2021.csv
2025-06-25 23:16:07,429 - DEBUG :: file_handler.py :: file_dump (273) :: population.nb_menages_2021 -> results saved to : 'data/imports/population/population.nb_menages_2021_population_nb_menages_2021_td_men1_2021.csv'
{'name': 'population_nb_menages_2021_td_men1_2021', 'storage_info': {'location': 'data/imports/population', 'format': 'csv', 'file_name': 'population.nb_menages_2021_population_nb_menages_2021_td_men1_2021.csv', 'encoding': 'utf-8'}, 'load_to_bronze': True, 'success': True}


# Dump intermédiaire

Sauvegarde locale de tous les artefacts extraits du zip

In [20]:
preprocess_metadata = handler.dump_metadata(
    model = model,
    operation = OperationType.PREPROCESS,
    start_time = start_time,
    complete = True,
    errors = 0,
    artifacts = artifacts,
    pages = []
)

2025-06-25 23:15:54,282 - DEBUG :: file_handler.py :: dump (130) :: dumping: data/imports/population/population.nb_menages_2021_metadata_preprocess.json
2025-06-25 23:15:54,285 - DEBUG :: file_handler.py :: file_dump (273) :: population.nb_menages_2021 -> results saved to : 'data/imports/population/population.nb_menages_2021_metadata_preprocess.json'
2025-06-25 23:15:54,286 - DEBUG :: file_handler.py :: dump_metadata (455) :: Metadata written in: 'data/imports/population/population.nb_menages_2021_metadata_preprocess.json'


# Traitement Pandas

Import des CSV en Dataframes et chargement en tables Bronze

In [21]:
from pathlib import Path

for artifact in artifacts:

    base_path = Path( artifact.storage_info.location )
    filepath = base_path / artifact.storage_info.file_name

    df = pd.read_csv(
        filepath,
        sep = ';',
        engine = 'python'
        )

    dataframes[ artifact.name ] = df


KeyboardInterrupt: 

In [ ]:
print("Loaded dataframes:")
last_key = ''
for key in dataframes.keys():
    last_key = key
    print(key)

dataframes[last_key].head()

Loaded dataframes:
population_nb_menages_2021_td_men1_2021


,NIVGEO,CODGEO,LIBGEO,NPERC,CS2_24,NB
0,COM,01001,L'Abergement-Clémenciat,3,46,9.687066
1,COM,01001,L'Abergement-Clémenciat,2,61,4.801620
2,COM,01001,L'Abergement-Clémenciat,5,32,4.928460
3,COM,01001,L'Abergement-Clémenciat,2,56,4.801620
4,COM,01001,L'Abergement-Clémenciat,4,56,4.885812


In [ ]:
from dotenv import dotenv_values
import sqlalchemy
from sqlalchemy import text

# prepare db client
vals = dotenv_values()

conn_str = "postgresql://{}:{}@{}:{}/{}".format(
    vals["PG_DB_USER"],
    vals["PG_DB_PWD"],
    vals["PG_DB_HOST"],
    vals["PG_DB_PORT"],
    vals["PG_DB_NAME"]
)

dbengine = sqlalchemy.create_engine(conn_str)

In [ ]:
# insert all to bronze
# make the final table name lowercase to avoid issues in Postgre

for name, dataframe in dataframes.items():

    query_str = f"DROP TABLE IF EXISTS bronze.{name} CASCADE"

    # dropping existing table with cascade
    with dbengine.connect() as con:
        print(f"Dropping if exists: {name}")
        result = con.execute(text(query_str))
        con.commit()

    print(f"Inserting DataFrame {name}")
    dataframe.to_sql(
        name = name,
        con = dbengine,
        schema = 'bronze',
        index = True,
        if_exists = 'replace'
    )

Dropping if exists: emploi_ds_rp_navettes_princ_metadata
Inserting DataFrame emploi_ds_rp_navettes_princ_metadata
Dropping if exists: emploi_deplacement_domicile_travail_ds_rp_navettes_princ_data
Inserting DataFrame emploi_deplacement_domicile_travail_ds_rp_navettes_princ_data
